# ViEWS prediction competition 
## ARIMA Code and Results 

## Author: Iris Malone

# Load Packages

In [1]:
import tensorflow as tf
from tensorflow.python.keras.layers import LSTM
from tensorflow.python.keras.layers import Input, Dense, Dropout, RepeatVector, TimeDistributed, Activation, Bidirectional
from tensorflow.python.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

#plots
from plotnine import ggplot, aes
from plotnine import options
import plotnine as p9
from matplotlib import pyplot as plt

#random number generators since rnn is stochastic
from random import seed

In [2]:
import sys
import logging
import os
import pandas as pd
import numpy as np
import math
import csv
from datetime import datetime
from joblib import dump, load

# Import any other sklearn estimator you would like. 
# See https://scikit-learn.org/stable/user_guide.html#user-guide
#from sklearn.ensemble import RandomForestRegressor

from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler

from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
 
    
#import views package

import views
from views import Period, Model, Downsampling
from views.utils.data import assign_into_df
from views.apps.transforms import lib as translib
from views.apps.evaluation import lib as evallib, feature_importance as fi
from views.apps.model import api
from views.apps.extras import extras


In [3]:

logging.basicConfig(
    #level=logging.DEBUG,
    level=logging.INFO, # uncomment this and comment debug above for less yelling in red
    format=views.config.LOGFMT,
)

In [4]:
# Do you wish to fetch the latest public data? If so, change False to True and run this cell
# Cells below will fail if this is not run if you haven't imported data yourself yet.
if False:
    path_zip = views.apps.data.public.fetch_latest_zip_from_website(path_dir_destination=views.DIR_SCRATCH)
    views.apps.data.public.import_tables_and_geoms(tables=views.TABLES, geometries=views.GEOMETRIES, path_zip=path_zip)

In [5]:
# Enter the level of analysis, and whether to predict Africa-only.
level = "cm"
pred_africa = True

In [6]:
model_path = "./models/{sub}"
out_paths = {
    "evaluation": model_path.format(sub="evaluation"),
    "features": model_path.format(sub="features")
}
for k, v in out_paths.items():
    if not os.path.isdir(v):
        os.makedirs(v)

In [7]:
if level == "pgm":
    dataset = views.DATASETS["pgm_africa_imp_0"]
elif level == "cm":
    dataset = views.DATASETS["cm_global_imp_0"]
df = dataset.df

In [8]:
df.reset_index(inplace=True)

# Clean Data

In [9]:
df["time"] = df["month_id"]
df["gwno"] = df["country_id"]

df = df.dropna(subset=['country_id'], how='all')
df = df.set_index(["month_id", "country_id"]).sort_index()

[2020-11-24 17:37:14,574] - numexpr.utils:141 - INFO - NumExpr defaulting to 8 threads.


In [10]:
df.head()

acled_count_ns  acled_count_os  acled_count_pr  \
month_id country_id                                                   
1        1                      0.0             0.0             0.0   
         2                      0.0             0.0             0.0   
         3                      0.0             0.0             0.0   
         4                      0.0             0.0             0.0   
         5                      0.0             0.0             0.0   

                     acled_count_sb  acled_dummy_ns  acled_dummy_os  \
month_id country_id                                                   
1        1                      0.0               0               0   
         2                      0.0               0               0   
         3                      0.0               0               0   
         4                      0.0               0               0   
         5                      0.0               0               0   

                     acled_dummy_pr  acled_dummy_sb  cdum_1  cdum_10  ...  \
month_id country_id                                                   ...   
1        1                        0               0       1        0  ...   
         2                        0               0       0        0  ...   
         3                        0               0       0        0  ...   
         4                        0               0       0        0  ...   
         5                        0               0       0        0  ...   

                     wdi_vc_idp_nwcv  wdi_vc_idp_nwds  wdi_vc_idp_tocv  \
month_id country_id                                                      
1        1              3.600000e+04            200.0          23000.0   
         2              2.667522e+05            300.0           2100.0   
         3              7.500000e+04             29.0         164000.0   
         4              1.672000e+06           4700.0         197000.0   
         5              1.500000e+02           5300.0           2100.0   

                     wdi_vc_ihr_psrc_fe_p5  wdi_vc_ihr_psrc_ma_p5  \
month_id country_id                                                 
1        1                             7.5                   30.3   
         2                             5.8                   11.9   
         3                             4.0                   36.6   
         4                             1.9                   22.9   
         5                             0.0                   10.0   

                     wdi_vc_ihr_psrc_p5  wdi_vc_pkp_totl_un  year  time  gwno  
month_id country_id                                                            
1        1                         12.1               141.0  1980     1     1  
         2                         14.4              9057.0  1980     1     2  
         3                          9.5               871.0  1980     1     3  
         4                         12.5              8536.0  1980     1     4  
         5                          8.7              1216.0  1980     1     5  

[5 rows x 3159 columns]

## Define Time Periods

In [11]:
# Define development period
# Keeping periods in a list lets us easily expand this as the 
# updated data becomes available
period_calib = {'train_start': 121,   # 1990-01
          'train_end': 408,     # 2013.12
          'predict_start': 409, # 2014.01
          'predict_end': 444}   # 2016.12

print(period_calib)

period_test = {'train_start': 121,   # 1990-01
          'train_end': 444,     # 2016.12
          'predict_start': 445, # 2017.01
          'predict_end': 480}   # 2019.12

print(period_test)

period_task1 = {'train_start': 121,   # 1990-01
          'train_end': 481,     # 2019.01
          'predict_start': 482, # 2020.02
          'predict_end': 488}   # 2020.08

print(period_task1)

periods = [period_calib, period_test, period_task1]


{'train_start': 121, 'train_end': 408, 'predict_start': 409, 'predict_end': 444}
{'train_start': 121, 'train_end': 444, 'predict_start': 445, 'predict_end': 480}
{'train_start': 121, 'train_end': 481, 'predict_start': 482, 'predict_end': 488}


## Write Model Functions

In [27]:
scaler = MinMaxScaler()

#create training and test based on period predictions

def partition_dataset(dataset, period, look_back):
    #create delta/first diff
    from views.apps.transforms import lib as translib
    dataset['s_outcome'] = translib.delta(dataset['ln_ged_best_sb'], time=1)

    train_df = dataset[dataset['time'].between(period['train_start'] , period['train_end']-look_back, inclusive=True)].dropna()
    test_df = dataset[dataset['time'].between(period['predict_start'] - look_back, period['predict_end'], inclusive=True)].dropna()
    #print(train_df)
    #print(test_df) 
    #identify outcome of interest
    train_df = train_df[['s_outcome']]
    test_df = test_df[['s_outcome']]   
    
    return np.array(train_df), np.array(test_df)

def partition_delta(countryname, period, dataset, look_back):

    #s_actual = df.loc[:, self.model.col_outcome].copy()

    #if self.model.delta_outcome:
    #    s_actual = translib.delta(s_actual, time=step)
    
    dataset['s_outcome1'] = translib.delta(dataset['ln_ged_best_sb'], time=1)
    dataset['s_outcome2'] = translib.delta(dataset['ln_ged_best_sb'], time=2)
    dataset['s_outcome3'] = translib.delta(dataset['ln_ged_best_sb'], time=3)
    dataset['s_outcome4'] = translib.delta(dataset['ln_ged_best_sb'], time=4)
    dataset['s_outcome5'] = translib.delta(dataset['ln_ged_best_sb'], time=5)
    dataset['s_outcome6'] = translib.delta(dataset['ln_ged_best_sb'], time=6)
    dataset['s_outcome7'] = translib.delta(dataset['ln_ged_best_sb'], time=7)
    #translib.delta(dataset['ln_ged_best_sb'], time=1)
    
    train_df = dataset[dataset['time'].between(period['train_start'] , period['train_end']-look_back)].dropna()
    test_df = dataset[dataset['time'].between(period['predict_start'] - look_back , period['predict_end'])].dropna()
     
    #identify outcome of interest
    train_df = train_df[['s_outcome1', 's_outcome2', 's_outcome3', 's_outcome4', 's_outcome5', 's_outcome6', 's_outcome7']]
    test_df = test_df[['s_outcome1', 's_outcome2', 's_outcome3', 's_outcome4', 's_outcome5', 's_outcome6', 's_outcome7']]

    return train_df, test_df
    
     
# split a univariate sequence into samples
def create_dataset(dataset, look_back, look_forward):
    X, y = list(), list()
    for i in range(len(dataset)):
        # find the end of this pattern
        end_ix = i + look_back
        out_end_ix = end_ix + look_forward
        # check if we are beyond the sequence
        if out_end_ix > len(dataset):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = dataset[i:end_ix], dataset[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)
 
def make_forecasts(period, X_train, y_train, test_df, X_test, y_test, look_forward):
    """FORECASTING""" 
    print('Forecasting...')
    forecasts = []
    futures = []
    history = [x for x in X_train]
    #print(history)
    for t in range(len(X_test)):
        model = ARIMA(history, order=(6, 1, 1))
        model_fit = model.fit(disp=0)
        output = model_fit.forecast(steps=look_forward)
        yhat = output[0]
        #print(yhat)
        forecasts.append(yhat)
    return forecasts  

def eval_model(countryname, X_train, y_train, X_test, y_test, forecasts, look_back, look_forward):
    """EVALUATING THE MODEL"""
    print('Evaluating...')
    testScore_values = []
    testScore_valuespercountry = []
    for i in range(look_forward):
        #print(type(y_test))
        s_actual = [row[i] for row in y_test]
        #print(s_actual)
        s_prediction = [forecast[i] for forecast in forecasts]
        #print(s_prediction)
        scores = {}
        scores["Country"] = countryname.ravel().tolist()[0]
        scores["Step"] = 't+%d' % ((i))
        scores["mse"] = evallib.mean_squared_error(actuals=s_actual, preds=s_prediction)
        scores["mae"] = evallib.mean_absolute_error(actuals=s_actual, preds=s_prediction)
        scores["r2"] = evallib.r2_score(actuals=s_actual, preds=s_prediction)
        scores["tadda_score"] = evallib.tadda_score(y_deltas=np.array(s_actual), f_deltas=np.array(s_prediction), epsilon=0.048)
        #print(scores)
        
        testScore_values.append(scores)
    testScore_valuespercountry.append(testScore_values)
    return testScore_valuespercountry

def eval_delta_forecasts(period, countryname, gwno, train_step, test_step, y_test, forecasts, look_back, look_forward):
    print('Evaluating delta multi-step forecasts...')
    storage_pred2 = []
    storage_actual2 = []
    testScore_values = []
    testScore_valuespercountry = []
    for i in range(len(forecasts)):
        
        yaxis = forecasts[i:][0]  
        storage_pred = (yaxis[0], yaxis[0] + yaxis[1], yaxis[0] + yaxis[1] + yaxis[2] ,yaxis[0] + yaxis[1] + yaxis[2] + yaxis[3],yaxis[0] + yaxis[2] + yaxis[3] + yaxis[4], yaxis[0] + yaxis[2] + yaxis[3] + yaxis[4] + yaxis[5], yaxis[0] + yaxis[2] + yaxis[3] +yaxis[4] + yaxis[5] + yaxis[6]) 
        storage_pred = list(storage_pred)
        storage_pred2.append(storage_pred)

    for index, row in test_step.iterrows(): 
        #yaxis = test_step[i:, 0]
        #print(yaxis)
        storage_actual = (row.loc['s_outcome1'], row.loc['s_outcome2'], row.loc['s_outcome3'], row.loc['s_outcome4'], row.loc['s_outcome5'], row.loc['s_outcome6'], row.loc['s_outcome7'])
        storage_actual = list(storage_actual)
        storage_actual2.append(storage_actual)
        #print(storage_actual2)
        
    for i in range(look_forward):
        s_actual = [item[i] for item in storage_actual2]
        s_prediction = [item[i] for item in storage_pred2]
        
        scores = {}
        scores["country"] = countryname.ravel().tolist()[0]
        scores["country_id"] = gwno.ravel().tolist()[0]
        scores["step"] = 't+%d' % ((i))
        scores["mse"] = evallib.mean_squared_error(actuals=s_actual, preds=s_prediction)
        scores["mae"] = evallib.mean_absolute_error(actuals=s_actual, preds=s_prediction)
        scores["r2"] = evallib.r2_score(actuals=s_actual, preds=s_prediction)
        scores["tadda_score"] = evallib.tadda_score(y_deltas=np.array(s_actual), f_deltas=np.array(s_prediction), epsilon=0.048)
        print(scores)
        # calculate root mean squared error for scaled data
        
        testScore_values.append(scores)
    testScore_valuespercountry.append(testScore_values)
    return testScore_valuespercountry

def export_results(modelname, period_test, look_back, look_forward, scoresglobal, storagepredictionsglobal, scoresdeltaglobal):
    print('Exporting results...')

    #Write .csv file with evaluation metrics, one per country-step
    flattenscores_list = [item for sublist in scoresglobal for item in sublist]
    
    flattenscores_list_final = [item for sublist in flattenscores_list for item in sublist]
    keys = flattenscores_list_final[0].keys()
    with open('country.csv', 'w') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(flattenscores_list_final)
    
    path_out = os.path.join(
        out_paths["evaluation"], 
        f"{modelname}_lookback{look_back}_countryscores.csv"
        )
    with open(path_out, "w") as f:
        dict_writer = csv.DictWriter(f, keys)
        dict_writer.writeheader()
        dict_writer.writerows(flattenscores_list_final)
    print(f"Wrote country-level error tables to {path_out}.")
 
    #Write .csv file with evaluation metrics, one per country-step
    flattenscores_listdelta = [item for sublist in scoresdeltaglobal for item in sublist]
    
    flattenscores_list_final = [item for sublist in flattenscores_listdelta for item in sublist]
    keys = flattenscores_list_final[0].keys()
    with open('country.csv', 'w') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(flattenscores_list_final)
    
    path_out = os.path.join(
        out_paths["evaluation"], 
        f"{modelname}_lookback{look_back}_countryscores_delta.csv"
        )
    with open(path_out, "w") as f:
        dict_writer = csv.DictWriter(f, keys)
        dict_writer.writeheader()
        dict_writer.writerows(flattenscores_list_final)
    print(f"Wrote country-level delta error tables to {path_out}.")
    
    #Write .tex file with evaluation metrics, averaged over countries    
    mselist = []
    mse2 = []
    scores2 = []
    sumscores = []
    scores = {
            "Step":[], 
            "MSE":[], 
            "MAE":[],
            "R2":[],
            "tadda_score":[]
        }
    
    for i in range(look_forward):
        evalmetrics = [a[i] for a in flattenscores_listdelta]
        mse = [d['mse'] for d in evalmetrics]
        mae = [d['mae'] for d in evalmetrics]
        r2 = [d['r2'] for d in evalmetrics]
        tadda = [d['tadda_score'] for d in evalmetrics]

        scores["Step"].append('t+%d' % ((i)))
        scores["MSE"].append(np.mean(mse))
        scores["MAE"].append(np.mean(mae))
        scores["R2"].append(np.mean(r2))
        scores["tadda_score"].append(np.mean(tadda))
        #print(scores)
        scores_summary = scores
        #print(scores2)
        
    out = pd.DataFrame([v for v in scores_summary.values()],
                    index = [k for k in scores_summary.keys()])
    #print(out)
    out_transpose = out.T
    tex = out_transpose.to_latex(index=False)
    
    now = datetime.now().strftime("%Y/%m/%d %H:%M:%S")
    meta = f"""
        %Output created by LSTM Multi-Step.ipynb.
        %Produced on {now}, written to {out_paths["evaluation"]}.
        \\
        """
    tex = meta + tex
    path_out = os.path.join(
        out_paths["evaluation"], 
        f"{modelname}_lookback{look_back}_delta_scores.tex"
        )
    with open(path_out, "w") as f:
        f.write(tex)
    print(f"Wrote scores table to {path_out}.")
#Write .tex file with evaluation metrics, averaged over countries    
    mselist = []
    mse2 = []
    scores2 = []
    sumscores = []
    scores = {
            "Step":[], 
            "MSE":[], 
            "MAE":[],
            "R2":[],
            "tadda_score":[]
        }
    
    for i in range(look_forward):
        evalmetrics = [a[i] for a in flattenscores_list]
        mse = [d['mse'] for d in evalmetrics]
        mae = [d['mae'] for d in evalmetrics]
        r2 = [d['r2'] for d in evalmetrics]
        tadda = [d['tadda_score'] for d in evalmetrics]

        scores["Step"].append('t+%d' % ((i)))
        scores["MSE"].append(np.mean(mse))
        scores["MAE"].append(np.mean(mae))
        scores["R2"].append(np.mean(r2))
        scores["tadda_score"].append(np.mean(tadda))
        #print(scores)
        scores_summary = scores
        #print(scores2)
        
    out = pd.DataFrame([v for v in scores_summary.values()],
                    index = [k for k in scores_summary.keys()])
    #print(out)
    out_transpose = out.T
    tex = out_transpose.to_latex(index=False)
    
    now = datetime.now().strftime("%Y/%m/%d %H:%M:%S")
    meta = f"""
        %Output created by LSTM Multi-Step.ipynb.
        %Produced on {now}, written to {out_paths["evaluation"]}.
        \\
        """
    tex = meta + tex
    path_out = os.path.join(
        out_paths["evaluation"], 
        f"{modelname}_lookback{look_back}_scores.tex"
        )
    with open(path_out, "w") as f:
        f.write(tex)
    print(f"Wrote scores table to {path_out}.")


In [28]:
dfafr = df[(df.in_africa==1)]

#create unique list of names
gwnonames = dfafr.gwno.unique()
    
#create a data frame dictionary to store data frame for each unique country
gwno_dict = {elem : pd.DataFrame for elem in gwnonames}

for key in gwno_dict.keys():
    gwno_dict[key] = dfafr[:][dfafr.gwno == key]
    
#model needs to be global
forecasts = []
predictions = []
forecastsglobal = []
storagepredictionsglobal = []
scoresglobal = []
    
scores = []
scoresdelta = []
scoresdeltaglobal = []
futures = []
storage = []
actual = list()
predictions = []
for key in gwno_dict:
#for key in range(4, 6):
    forecasts = []
    history = []
    forecasts=[]
    modelname = "arima"
    dfgwno = pd.DataFrame(gwno_dict[key])
    dfgwno = dfgwno[['gwno','time','ln_ged_best_sb','country_name']]

    countryname = dfgwno.country_name.unique()
    gwno = dfgwno.gwno.unique()
    #dfgwno['ln_ged_best_sb'] = dfgwno['ln_ged_best_sb']  + 0.00001*np.random.rand(len(dfgwno), 1)
    print(countryname)
    look_back=1
    look_forward=7
    train_df, test_df = partition_dataset(dfgwno, period_test, look_back)
    #train_df = train_df +0.00001*np.random.rand(1000, 1)
    train_step, test_step = partition_delta(countryname, period_test, dfgwno, look_back)        
        
    test_step = test_step.iloc[look_back: -look_forward+1]
    train_df = train_df+0.00001*np.random.rand(len(train_df), 1)
    test_df = test_df+0.00001*np.random.rand(len(test_df), 1)

    X_train, y_train = create_dataset(train_df, look_back, look_forward)
    # Create the data to test our model on:
    X_test, y_test = create_dataset(test_df, look_back, look_forward)   
    #print(X_train)
    #print(X_train)
    forecasts = make_forecasts(period_test, X_train, y_train, test_df, X_test, y_test, look_forward)
    #print(forecasts) 
    scores = eval_model(countryname, X_train, y_train, X_test, y_test, forecasts, look_back, look_forward)
    #print(scores)
    scoresglobal.append(scores)
    scoresdelta = eval_delta_forecasts(period_test, countryname, gwno, train_step, test_step, y_test, forecasts, look_back, look_forward)
    print(scoresdelta)
    scoresdeltaglobal.append(scoresdelta)
export_results(modelname, period_test, look_back, look_forward, scoresglobal, storagepredictionsglobal, scoresdeltaglobal)
         #obs = y_test[t]
            #history.append(obs)
            #print('predicted=%f, expected=%f' % (yhat, obs))
        #except:
            #pass
    #print('Plotting forecasts...')
    #if dfgwno.corr() == 1:
    #    error = 0
    #else:
    #error = mean_squared_error(yhat, obs)
    #print('Test MSE: %.2f' % error)
# plot
#print(history[len(X_train):])
    #history2 = np.concatenate(history).ravel().tolist() 
    #plt.plot(history2[len(X_train):], color="blue")
    #plt.plot(predictions, color='red')
    #plt.show()
    
    

['Cape Verde']
Forecasting...


/Users/irismalone/.local/share/virtualenvs/irismalone-z6vzrJQw/lib/python3.8/site-packages/statsmodels/base/model.py:547: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
/Users/irismalone/.local/share/virtualenvs/irismalone-z6vzrJQw/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
/Users/irismalone/.local/share/virtualenvs/irismalone-z6vzrJQw/lib/python3.8/site-packages/statsmodels/base/model.py:547: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
/Users/irismalone/.local/share/virtualenvs/irismalone-z6vzrJQw/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
/Users/irismalone/.local/share/virtualenvs/irismalone-z6vzrJQw/lib/python3.8/site-packages/statsmodels/base/model.py:547: HessianInversionWarning: Inverting hessian fai

/Users/irismalone/.local/share/virtualenvs/irismalone-z6vzrJQw/lib/python3.8/site-packages/statsmodels/base/model.py:547: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
/Users/irismalone/.local/share/virtualenvs/irismalone-z6vzrJQw/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
/Users/irismalone/.local/share/virtualenvs/irismalone-z6vzrJQw/lib/python3.8/site-packages/statsmodels/base/model.py:547: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
/Users/irismalone/.local/share/virtualenvs/irismalone-z6vzrJQw/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
/Users/irismalone/.local/share/virtualenvs/irismalone-z6vzrJQw/lib/python3.8/site-packages/statsmodels/base/model.py:547: HessianInversionWarning: Inverting hessian fai

Evaluating...
Evaluating delta multi-step forecasts...
{'country': 'Cape Verde', 'country_id': 40, 'step': 't+0', 'mse': 6.307828640790679e-12, 'mae': 2.5115390980015973e-06, 'r2': 0.0, 'tadda_score': 2.5115390980015973e-06}
{'country': 'Cape Verde', 'country_id': 40, 'step': 't+1', 'mse': 3.0589131345373216e-11, 'mae': 5.530744194534151e-06, 'r2': 0.0, 'tadda_score': 5.530744194534151e-06}
{'country': 'Cape Verde', 'country_id': 40, 'step': 't+2', 'mse': 7.422907854568793e-11, 'mae': 8.615629898370051e-06, 'r2': 0.0, 'tadda_score': 8.615629898370051e-06}
{'country': 'Cape Verde', 'country_id': 40, 'step': 't+3', 'mse': 1.2694464933461874e-10, 'mae': 1.1266971613287168e-05, 'r2': 0.0, 'tadda_score': 1.1266971613287168e-05}
{'country': 'Cape Verde', 'country_id': 40, 'step': 't+4', 'mse': 1.240820676808462e-10, 'mae': 1.1139213063805102e-05, 'r2': 0.0, 'tadda_score': 1.1139213063805102e-05}
{'country': 'Cape Verde', 'country_id': 40, 'step': 't+5', 'mse': 1.9266171055708578e-10, 'mae': 

/Users/irismalone/.local/share/virtualenvs/irismalone-z6vzrJQw/lib/python3.8/site-packages/statsmodels/base/model.py:547: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
/Users/irismalone/.local/share/virtualenvs/irismalone-z6vzrJQw/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
/Users/irismalone/.local/share/virtualenvs/irismalone-z6vzrJQw/lib/python3.8/site-packages/statsmodels/base/model.py:547: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
/Users/irismalone/.local/share/virtualenvs/irismalone-z6vzrJQw/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
/Users/irismalone/.local/share/virtualenvs/irismalone-z6vzrJQw/lib/python3.8/site-packages/statsmodels/base/model.py:547: HessianInversionWarning: Inverting hessian fai

...

Evaluating...
Evaluating delta multi-step forecasts...
{'country': 'Uganda', 'country_id': 235, 'step': 't+0', 'mse': 0.0013883967325954171, 'mae': 0.037261196070381544, 'r2': 0.0, 'tadda_score': 0.037261196070381544}
{'country': 'Uganda', 'country_id': 235, 'step': 't+1', 'mse': 0.007308376318803359, 'mae': 0.08548904209782301, 'r2': 0.0, 'tadda_score': 0.08548904209782301}
{'country': 'Uganda', 'country_id': 235, 'step': 't+2', 'mse': 0.01988822155574058, 'mae': 0.14102560602862368, 'r2': 0.0, 'tadda_score': 0.14102560602862368}
{'country': 'Uganda', 'country_id': 235, 'step': 't+3', 'mse': 0.04244826927765125, 'mae': 0.20602977764791966, 'r2': 0.0, 'tadda_score': 0.20602977764791966}
{'country': 'Uganda', 'country_id': 235, 'step': 't+4', 'mse': 0.052602045473332276, 'mae': 0.22935135812401958, 'r2': 0.0, 'tadda_score': 0.22935135812401958}
{'country': 'Uganda', 'country_id': 235, 'step': 't+5', 'mse': 0.09593600189677337, 'mae': 0.3097353739836207, 'r2': 0.0, 'tadda_score': 0.30973

/Users/irismalone/.local/share/virtualenvs/irismalone-z6vzrJQw/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:701: RuntimeWarning: overflow encountered in exp
/Users/irismalone/.local/share/virtualenvs/irismalone-z6vzrJQw/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:701: RuntimeWarning: invalid value encountered in true_divide
/Users/irismalone/.local/share/virtualenvs/irismalone-z6vzrJQw/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:702: RuntimeWarning: overflow encountered in exp
/Users/irismalone/.local/share/virtualenvs/irismalone-z6vzrJQw/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:702: RuntimeWarning: invalid value encountered in true_divide
/Users/irismalone/.local/share/virtualenvs/irismalone-z6vzrJQw/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:701: RuntimeWarning: overflow encountered in exp
/Users/irismalone/.local/share/virtualenvs/irismalone-z6vzrJQw/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:701: RuntimeWar

/Users/irismalone/.local/share/virtualenvs/irismalone-z6vzrJQw/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:701: RuntimeWarning: overflow encountered in exp
/Users/irismalone/.local/share/virtualenvs/irismalone-z6vzrJQw/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:701: RuntimeWarning: invalid value encountered in true_divide
/Users/irismalone/.local/share/virtualenvs/irismalone-z6vzrJQw/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:702: RuntimeWarning: overflow encountered in exp
/Users/irismalone/.local/share/virtualenvs/irismalone-z6vzrJQw/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:702: RuntimeWarning: invalid value encountered in true_divide
/Users/irismalone/.local/share/virtualenvs/irismalone-z6vzrJQw/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:701: RuntimeWarning: overflow encountered in exp
/Users/irismalone/.local/share/virtualenvs/irismalone-z6vzrJQw/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:701: RuntimeWar

/Users/irismalone/.local/share/virtualenvs/irismalone-z6vzrJQw/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:701: RuntimeWarning: overflow encountered in exp
/Users/irismalone/.local/share/virtualenvs/irismalone-z6vzrJQw/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:701: RuntimeWarning: invalid value encountered in true_divide
/Users/irismalone/.local/share/virtualenvs/irismalone-z6vzrJQw/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:702: RuntimeWarning: overflow encountered in exp
/Users/irismalone/.local/share/virtualenvs/irismalone-z6vzrJQw/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:702: RuntimeWarning: invalid value encountered in true_divide
/Users/irismalone/.local/share/virtualenvs/irismalone-z6vzrJQw/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:701: RuntimeWarning: overflow encountered in exp
/Users/irismalone/.local/share/virtualenvs/irismalone-z6vzrJQw/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:701: RuntimeWar

Evaluating...
Evaluating delta multi-step forecasts...
{'country': 'Mauritania', 'country_id': 244, 'step': 't+0', 'mse': 0.0015486166857839115, 'mae': 0.03935246734048465, 'r2': 0.0, 'tadda_score': 0.03935246734048465}
{'country': 'Mauritania', 'country_id': 244, 'step': 't+1', 'mse': 0.006035880698350398, 'mae': 0.07769093060551144, 'r2': 0.0, 'tadda_score': 0.07769093060551144}
{'country': 'Mauritania', 'country_id': 244, 'step': 't+2', 'mse': 0.013850539583029075, 'mae': 0.11768831540568965, 'r2': 0.0, 'tadda_score': 0.11768831540568965}
{'country': 'Mauritania', 'country_id': 244, 'step': 't+3', 'mse': 0.025817061794846566, 'mae': 0.16067688631177351, 'r2': 0.0, 'tadda_score': 0.16067688631177351}
{'country': 'Mauritania', 'country_id': 244, 'step': 't+4', 'mse': 0.027126893316096894, 'mae': 0.16470243870719367, 'r2': 0.0, 'tadda_score': 0.16470243870719367}
{'country': 'Mauritania', 'country_id': 244, 'step': 't+5', 'mse': 0.04538918067954197, 'mae': 0.21304736722039527, 'r2': 0.

# Calibration Period

In [ ]:
np.random.seed(1)
tf.random.set_seed(1)
model = Sequential()
summary_model(dataset=df,
              modelname = "Vanilla",
              pred_africa=True,
              period=period_calib, 
              epochs=50, 
              batch_size=64, 
              look_back=4, 
              look_forward=7,
              num_neurons=2, 
              val_split=0.2)

# Test Period

In [ ]:
np.random.seed(1)
tf.random.set_seed(1)
model = Sequential()
summary_model(dataset=df,
              modelname = "Vanilla",
              pred_africa=True,
              period=period_test, 
              epochs=50, 
              batch_size=64, 
              look_back=4, 
              look_forward=7,
              num_neurons=2, 
              val_split=0.2)

# Single Forecast for Aug 2020

In [ ]:
scaler = MinMaxScaler()

#create training and test based on period predictions

def partition_dataset(dataset, period, look_back):
    #create delta/first diff
    from views.apps.transforms import lib as translib
    dataset['s_outcome'] = translib.delta(dataset['ln_ged_best_sb'], time=1)

    train_df = dataset[dataset['time'].between(period['train_start'] , period['train_end']-look_back, inclusive=True)].dropna()
    test_df = dataset[dataset['time'].between(period['predict_start'] - look_back, period['predict_end'], inclusive=True)].dropna()
    #print(train_df)
    #print(test_df) 
    #identify outcome of interest
    train_df = train_df[['s_outcome']]
    test_df = test_df[['s_outcome']]   
    
    return np.array(train_df), np.array(test_df)

def partition_delta(countryname, period, dataset, look_back):

    #s_actual = df.loc[:, self.model.col_outcome].copy()

    #if self.model.delta_outcome:
    #    s_actual = translib.delta(s_actual, time=step)
    
    dataset['s_outcome1'] = translib.delta(dataset['ln_ged_best_sb'], time=1)
    dataset['s_outcome2'] = translib.delta(dataset['ln_ged_best_sb'], time=2)
    dataset['s_outcome3'] = translib.delta(dataset['ln_ged_best_sb'], time=3)
    dataset['s_outcome4'] = translib.delta(dataset['ln_ged_best_sb'], time=4)
    dataset['s_outcome5'] = translib.delta(dataset['ln_ged_best_sb'], time=5)
    dataset['s_outcome6'] = translib.delta(dataset['ln_ged_best_sb'], time=6)
    dataset['s_outcome7'] = translib.delta(dataset['ln_ged_best_sb'], time=7)
    #translib.delta(dataset['ln_ged_best_sb'], time=1)
    
    train_df = dataset[dataset['time'].between(period['train_start'] , period['train_end']-look_back)].dropna()
    test_df = dataset[dataset['time'].between(period['predict_start'] - look_back , period['predict_end'])].dropna()
     
    #identify outcome of interest
    train_df = train_df[['s_outcome1', 's_outcome2', 's_outcome3', 's_outcome4', 's_outcome5', 's_outcome6', 's_outcome7']]
    test_df = test_df[['s_outcome1', 's_outcome2', 's_outcome3', 's_outcome4', 's_outcome5', 's_outcome6', 's_outcome7']]

    return train_df, test_df
    
     
# split a univariate sequence into samples
def create_dataset(dataset, look_back, look_forward):
    X, y = list(), list()
    for i in range(len(dataset)):
        # find the end of this pattern
        end_ix = i + look_back
        out_end_ix = end_ix + look_forward
        # check if we are beyond the sequence
        if out_end_ix > len(dataset):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = dataset[i:end_ix], dataset[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)
 

#VANILLA LSTM
def create_vanilla_model(X_train, y_train, epochs, num_neurons, batch_size, val_split):
    """BUILDING THE MODEL""" 
    print('Building vanilla Model...')
    # simple early stopping
    es = EarlyStopping(monitor='loss', mode='min', verbose=1)

    model = Sequential()

    #Input Layers
    model.add(LSTM(units = num_neurons, input_shape = (X_train.shape[1], 1), activation="tanh"))

    #model.add(LSTM(units = num_neurons, input_shape = (X_train.shape[1], 1), activation="tanh", stateful=True))
    #model.add(TimeDistributed(Dense(1)))

    # Dropout Regularization - randomly removes 20\%, helps with overfitting
    model.add(Dropout(0.2))
    #model.add(Dense(4)) 
    
    #Hidden Layers
    #
    model.add(Dense(num_neurons, activation="tanh")) # New hidden layer with 2 params, sigmoid function to constrainw weights
    #model.add(Dense(num_neurons, activation="tanh"))
    
    model.add(Dropout(0.2))
              
    # Output Layers
    model.add(Dense(units = y_train.shape[1]))

    #model.add(Activation('relu'))
    #model.add(Dropout(0.2))

    # Compiling the model
    model.compile(optimizer = 'adam', loss = 'mean_squared_error')

    #Summarize the model
    model.summary()

    # Fitting the model to the training set
    #model.fit(X_train, y_train, epochs = epochs, batch_size = batch_size, validation_split=val_split, callbacks=[es])
    #remove callback, train for more epochs (passes through the data)
    model.fit(X_train, y_train, epochs = epochs, batch_size = batch_size, validation_split=val_split)

    return model


def make_forecasts(model, period, batch_size, X_train, y_train, test_df, X_test, y_test, look_back, look_forward):
    """FORECASTING""" 
    print('Forecasting...')
    forecasts = []
    futures = []

    for i in range(0, len(X_test)):
        X = X_test[i, 0:look_back]
        y = y_test[i, look_back:]  

        # forecast steps       
        forecast = forecast_lstm(model, X, look_back, batch_size)
        #print(forecast)
        # store the forecast
        forecasts.append(forecast)
    
    currentStep = forecast[-1:]
    #Make one-step predictions for data beyond test_set
    for i in range(look_forward):
        if i == 0:
            X = X.reshape(1, look_back, 1)
            yhat = forecast_lstm(model, X, look_back, batch_size)
            #print(yhat)
        else:
            x0_input = np.append(X, [currentStep[i-1]])
            x0_input = x0_input.reshape((1, look_back+1, 1))
            x_input = x0_input[:,1:]
            yhat = forecast_lstm(model, x_input, look_back, batch_size)
            currentStep = np.append(currentStep, yhat)        
        #print(period['predict_end'] + i - look_forward + 1)
        futures.append(yhat)
    #print(futures)
        
    return forecasts, futures

def forecast_lstm(model, X, look_back, batch_size): 
    #make predictions
    X = X.reshape(1, look_back, 1)
    
    testPredict = model.predict(X, batch_size)
    # convert to array
    return [x for x in testPredict[0, :]]

def eval_model(countryname, model, X_train, y_train, X_test, y_test, futures, forecasts, look_back, look_forward):
    """EVALUATING THE MODEL"""
    print('Evaluating...')
    testScore_values = []
    testScore_valuespercountry = []
    for i in range(look_forward):
        #print(type(y_test))
        s_actual = [row[i] for row in y_test]
        #print(s_actual)
        s_prediction = [forecast[i] for forecast in forecasts]
        #print(type(forecasts))
        #print(s_prediction)
        scores = {}
        scores["Country"] = countryname.ravel().tolist()[0]
        scores["Step"] = 't+%d' % ((i))
        scores["mse"] = evallib.mean_squared_error(actuals=s_actual, preds=s_prediction)
        scores["mae"] = evallib.mean_absolute_error(actuals=s_actual, preds=s_prediction)
        scores["r2"] = evallib.r2_score(actuals=s_actual, preds=s_prediction)
        scores["tadda_score"] = evallib.tadda_score(y_deltas=np.array(s_actual), f_deltas=np.array(s_prediction), epsilon=0.048)
        #scores["Future Prediction"] = flat_future2[i]
        #print(scores)
            # calculate root mean squared error for scaled data
        
        testScore_values.append(scores)
    testScore_valuespercountry.append(testScore_values)
    return testScore_valuespercountry

def plot_forecasts(period, test_df, y_test, forecasts, look_back, look_forward):
    print('Plotting forecasts...')
    """PLOTTING THE MODEL"""
    plt.plot(range(period['predict_start']- look_back, period['predict_start'] + len(test_df)- look_back), test_df, color='blue', label = 'Real Ln BD')
    # show the plot
    # plot the forecasts in red
    #print(test_df.shape)
    #print(len(forecasts))
    #print(len(tot))
    print(y_test.shape)
    for i in range(len(forecasts)):
        off_s = period['predict_start'] + i  
        off_e = off_s + len(forecasts[i]) 
        xaxis = [x for x in range(off_s, off_e)]
        yaxis = forecasts[i:][0]
        #print(xaxis)
        #print(yaxis)
        plt.plot(xaxis, yaxis, color='red')
    plt.title('Change in Ln Battle-Death Prediction')
    plt.xlabel('Time')
    plt.ylabel('Change in Ln Battle-Death')
    plt.legend()
    plt.show()
    
def store_forecasts(period, countryname, y_test, futures, forecasts, look_back, look_forward):
    print('Storing forecasts...')
    """STORING MODEL RESULTS"""
    storage =[]
    storagepredictions = []
    storagepredictionspercountry = []
    for i in range(len(forecasts)):
        off_s = period['predict_start'] + i 
        off_e = off_s + len(forecasts[i]) 
        #print(off_s)      
        xaxis = [x for x in range(off_s, off_e)]
        yaxis = forecasts[i:][0]

        storage = {}
        storage["Country"] = countryname.ravel().tolist()[0]
        storage["Month"] = off_s - 1
        #storage["Future Months"] = xaxis
        storage["Malone_LSTM_s1"] = yaxis[0]
        storage["Malone_LSTM_s2"] = yaxis[0] + yaxis[1]
        storage["Malone_LSTM_s3"] = yaxis[0] + yaxis[1] + yaxis[2]
        storage["Malone_LSTM_s4"] = yaxis[0] + yaxis[1] + yaxis[2] + yaxis[3]
        storage["Malone_LSTM_s5"] = yaxis[0] + yaxis[1] + yaxis[2] + yaxis[3] + yaxis[4]
        storage["Malone_LSTM_s6"] = yaxis[0] + yaxis[1] + yaxis[2] + yaxis[3] + yaxis[4] + yaxis[5]
        storage["Malone_LSTM_s7"] = yaxis[0] + yaxis[1] + yaxis[2] + yaxis[3] + yaxis[4] + yaxis[5] + yaxis[6]
        storagepredictions.append(storage)
    #storagepredictionspercountry.append(storagepredictions)
    #print(storagepredictionspercountry)
    for i in range(len(futures)):
        off_s = period['predict_end'] + i - look_forward + 1   
        off_e = off_s + len(futures[i]) 
        #print(off_s)
        
        xaxis = [x for x in range(off_s +1, off_e)]
        yaxis = futures[i:][0]
        #print(countryname)
        #print(xaxis)
        #print(yaxis)
        storage = {}
        storage["Country"] = countryname.ravel().tolist()[0]
        storage["Last Month of Data"] = off_s 
        #storage["Future Month"] = xaxis
        storage["Malone_LSTM_s1"] = yaxis[0]
        storage["Malone_LSTM_s2"] = yaxis[0] + yaxis[1]
        storage["Malone_LSTM_s3"] = yaxis[0] + yaxis[1] + yaxis[2]
        storage["Malone_LSTM_s4"] = yaxis[0] + yaxis[1] + yaxis[2] + yaxis[3]
        storage["Malone_LSTM_s5"] = yaxis[0] + yaxis[1] + yaxis[2] + yaxis[3] + yaxis[4]
        storage["Malone_LSTM_s6"] = yaxis[0] + yaxis[1] + yaxis[2] + yaxis[3] + yaxis[4] + yaxis[5]
        storage["Malone_LSTM_s7"] = yaxis[0] + yaxis[1] + yaxis[2] + yaxis[3] + yaxis[4] + yaxis[5] + yaxis[6]
        #print(storage)
        storagepredictions.append(storage)
    storagepredictionspercountry.append(storagepredictions)
    #print(storagepredictionspercountry)
    return storagepredictionspercountry



def store_forecasts(period, countryname, gwno, train_step, test_step, y_test, futures, forecasts, look_back, look_forward):
    print('Storing forecasts...')
    """STORING MODEL RESULTS"""
    storage =[]
    storagepredictions = []
    storagepredictionspercountry = []
    for i in range(1, len(futures)):
        off_s = period['predict_end'] + i - look_forward + 1   
        off_e = off_s + len(futures[i]) 
        #print(off_s)
        
        xaxis = [x for x in range(off_s +1, off_e)]
        yaxis = futures[i:][0]
        #print(countryname)
        #print(xaxis)
        #print(yaxis)
        storage = {}
        storage["country"] = countryname.ravel().tolist()[0]
        storage["country_id"] = gwno.ravel().tolist()[0]
        storage["Last Month of Data"] = off_s 
        #storage["Future Months"] = xaxis
        storage["Malone_LSTM_s1"] = yaxis[0]
        storage["Malone_LSTM_s2"] = yaxis[0] + yaxis[1]
        storage["Malone_LSTM_s3"] = yaxis[0] + yaxis[1] + yaxis[2]
        storage["Malone_LSTM_s4"] = yaxis[0] + yaxis[1] + yaxis[2] + yaxis[3]
        storage["Malone_LSTM_s5"] = yaxis[0] + yaxis[1] + yaxis[2] + yaxis[3] + yaxis[4]
        storage["Malone_LSTM_s6"] = yaxis[0] + yaxis[1] + yaxis[2] + yaxis[3] + yaxis[4] + yaxis[5]
        storage["Malone_LSTM_s7"] = yaxis[0] + yaxis[1] + yaxis[2] + yaxis[3] + yaxis[4] + yaxis[5] + yaxis[6]
        #print(storage)
    storagepredictions.append(storage)

    storagepredictionspercountry.append(storagepredictions)
    #print(storagepredictionspercountry)
    return storagepredictionspercountry

def export_results(modelname, period, batch_size, epochs, num_neurons, look_back, look_forward,  storagepredictionsglobal):
    print('Exporting results...')
    #Write Tables
    if period == period_calib:
        period_print = 'calibration'
    elif period == period_test:
        period_print = 'test'
    elif period == period_task1:
        period_print = 'task'
    
    #write country predictions
    flattenstorage_list = [item for sublist in storagepredictionsglobal for item in sublist]
    print(flattenstorage_list)
    flattenstorage_list_final = [item for sublist in flattenstorage_list for item in sublist]
    keys = flattenstorage_list_final[0].keys()
    with open('country.csv', 'w') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(flattenstorage_list_final)
   
    path_out = os.path.join(
        out_paths["evaluation"], 
        f"predictions_{modelname}_{period_print}_epochs{epochs}_batchsize{batch_size}_neurons{num_neurons}_lookback{look_back}_countryscores.csv"
        )
    with open(path_out, "w") as f:
        dict_writer = csv.DictWriter(f, keys)
        dict_writer.writeheader()
        dict_writer.writerows(flattenstorage_list_final)
    print(f"Wrote country-level prediction tables to {path_out}.")

def eval_delta_forecasts(period, countryname, gwno, train_step, test_step, y_test, futures, forecasts, look_back, look_forward):
    print('Evaluating delta multi-step forecasts...')
    storage_pred2 = []
    storage_actual2 = []
    testScore_values = []
    testScore_valuespercountry = []
    for i in range(len(forecasts)):
        
        yaxis = forecasts[i:][0]  
        storage_pred = (yaxis[0], yaxis[0] + yaxis[1], yaxis[0] + yaxis[1] + yaxis[2] ,yaxis[0] + yaxis[1] + yaxis[2] + yaxis[3],yaxis[0] + yaxis[2] + yaxis[3] + yaxis[4], yaxis[0] + yaxis[2] + yaxis[3] + yaxis[4] + yaxis[5], yaxis[0] + yaxis[2] + yaxis[3] +yaxis[4] + yaxis[5] + yaxis[6]) 
        storage_pred = list(storage_pred)
        storage_pred2.append(storage_pred)

    for index, row in test_step.iterrows(): 
        #yaxis = test_step[i:, 0]
        #print(yaxis)
        storage_actual = (row.loc['s_outcome1'], row.loc['s_outcome2'], row.loc['s_outcome3'], row.loc['s_outcome4'], row.loc['s_outcome5'], row.loc['s_outcome6'], row.loc['s_outcome7'])
        storage_actual = list(storage_actual)
        storage_actual2.append(storage_actual)
        
    for i in range(look_forward):
        s_actual = [item[i] for item in storage_actual2]
        s_prediction = [item[i] for item in storage_pred2]
        
        scores = {}
        scores["country"] = countryname.ravel().tolist()[0]
        scores["country_id"] = gwno.ravel().tolist()[0]
        scores["step"] = 't+%d' % ((i))
        scores["mse"] = evallib.mean_squared_error(actuals=s_actual, preds=s_prediction)
        scores["mae"] = evallib.mean_absolute_error(actuals=s_actual, preds=s_prediction)
        scores["r2"] = evallib.r2_score(actuals=s_actual, preds=s_prediction)
        scores["tadda_score"] = evallib.tadda_score(y_deltas=np.array(s_actual), f_deltas=np.array(s_prediction), epsilon=0.048)
        #print(scores)
        # calculate root mean squared error for scaled data
        
        testScore_values.append(scores)
    testScore_valuespercountry.append(testScore_values)
    return testScore_valuespercountry

In [ ]:

#list to store country-specific MSE

def summary_model(dataset, modelname, pred_africa, period, epochs, batch_size, look_back, look_forward, num_neurons, val_split):
    
    if pred_africa == False:
        dataset = dataset
    elif pred_africa == True:
        dataset = dataset[(dataset.in_africa==1)]

    if period == period_calib:
        period_print = 'calibration'
    elif period == period_test:
        period_print = 'test'
    elif period == period_task1:
        period_print = 'task1'    
    #create unique list of names
    gwnonames = dataset.gwno.unique()
    
    #create a data frame dictionary to store data frame for each unique country
    gwno_dict = {elem : pd.DataFrame for elem in gwnonames}

    for key in gwno_dict.keys():
        gwno_dict[key] = dataset[:][dataset.gwno == key]
    
    #model needs to be global
    forecasts = []
    predictions = []
    forecastsglobal = []
    storagepredictionsglobal = []
    scoresglobal = []
    
    scores = []
    scoresdelta = []
    scoresdeltaglobal = []
    futures = []
    storage = []
    for key in gwno_dict:
    #for key in range(8, 10):
        forecasts = []
        dfgwno = pd.DataFrame(gwno_dict[key])
        dfgwno = dfgwno[['gwno','time','ln_ged_best_sb','country_name']]
        countryname = dfgwno.country_name.unique()
        gwno = dfgwno.gwno.unique()

        print(countryname)
        #print(gwno)
        
        train_df, test_df = partition_dataset(dfgwno, period, look_back)
        #print(test_df)
        train_step, test_step = partition_delta(countryname, period, dfgwno, look_back)        
        
        test_step = test_step.iloc[look_back: -look_forward+1]
        X_train, y_train = create_dataset(train_df, look_back, look_forward)
    
        # reshape it [samples, time steps, features]
        X_train = np.reshape(X_train, (X_train.shape[0], look_back, 1))
    
        # Create the data to test our model on:
        X_test, y_test = create_dataset(test_df, look_back, look_forward)
        #print(y_test)
        # LSTM input shape: (samples, time steps, features)
        X_test = np.reshape(X_test, (X_test.shape[0], look_back, 1))
        
        #Create the model to predict:
        if modelname == "Vanilla":
            model = create_vanilla_model(X_train, y_train, epochs, num_neurons, batch_size, val_split)
        elif modelname == "Stateful":
            model = create_stateful_model(X_train, y_train, epochs, num_neurons, batch_size, val_split)
        elif modelname == "Bidirectional":
            model = create_bidirectional_model(X_train, y_train, epochs, num_neurons, batch_size, val_split)
        elif modelname == "Stacked":
            model = create_stacked_model(X_train, y_train, epochs, num_neurons, batch_size, val_split)
        
        #Make Multi-Step Predictions for Test Data 
        forecasts, futures = make_forecasts(model, period, batch_size, X_train, y_train, test_df, X_test, y_test, look_back, look_forward)

        forecastsglobal.append(forecasts)
        
        #Evaluate out-of-sample forecasting by different metrics

        #scores = eval_model(countryname,model, X_train, y_train, X_test, y_test, futures, forecasts, look_back, look_forward)
        # plot the forecasts in red
        #plot_forecasts(period, test_df, y_test, forecasts, look_back, look_forward)
        #scoresglobal.append(scores)
        #eval_delta_model(countryname, dataset, model, train_step, test_step, X_train, y_train, X_test, y_test, futures, forecasts, look_back, look_forward)
        # Store the Predictions for Write-Out
        storage = store_forecasts(period, countryname, gwno, train_step, test_step, y_test, futures, forecasts, look_back, look_forward)
    
        storagepredictionsglobal.append(storage)
        #scoresdelta = eval_delta_forecasts(period, countryname, gwno, train_step, test_step, y_test, futures, forecasts, look_back, look_forward)
        #scoresdeltaglobal.append(scoresdelta)
    export_results(modelname, period, batch_size, epochs, num_neurons, look_back, look_forward, storagepredictionsglobal)
    

# Forecast Oct 2020-March 2021

In [ ]:
period_task1 = {'train_start': 121,   # 1990-01
          'train_end': 481,     # 2020.01
          'predict_start': 482, # 2020.02
          'predict_end': 488}   # 2020.08

In [ ]:
np.random.seed(1)
tf.random.set_seed(1)
model = Sequential()
summary_model(dataset=df,
              modelname = "Vanilla",
              pred_africa=True,
              period=period_task1, 
              epochs=50, 
              batch_size=64, 
              look_back=4, 
              look_forward=7,
              num_neurons=2, 
              val_split=0.2)